In [62]:
import matplotlib.pyplot as plt
import moviepy.editor as mvp
import numpy as np
import os
import random
import time
import tqdm
from matplotlib.colors import to_rgb
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from pathlib import Path
from PIL import Image, ImageDraw, ImageTk

In [63]:
def clamp(lower, higher, val):
    if val < lower:
        val = lower
    elif val > higher:
        val = higher
    return val

In [64]:
def init_arrays(lowerBound, upperBound, h_in_blocks, w_in_blocks):
    curU = np.zeros((h_in_blocks, w_in_blocks))
    curV = np.zeros((h_in_blocks, w_in_blocks))
    for x in range(h_in_blocks):
        for y in range(w_in_blocks):
            curU[x, y] = round(random.uniform(lowerBound, upperBound), 5)
            curV[x, y] = round(random.uniform(lowerBound, upperBound), 5)
    return curU, curV

In [65]:
def get_input_illum(filename):
    vesicle_img = Image.open(filename)
    input_full_arr = np.array(vesicle_img)
    illum = input_full_arr[:, :, 2]
    # where blue, value 255
    return illum

In [66]:
def find_neighbours_sum(curU, x, y):
    global w_in_pixels, h_in_pixels
    neighbours_sum = 0
    neighbours = 4
    if x-1 >= 0:
        northN = curU[x-1, y]
        neighbours_sum += northN
    else:
        neighbours -= 1
        
    if y+1 < w_in_pixels:
        eastN = curU[x, y+1]
        neighbours_sum += eastN
    else:
        neighbours -= 1
        
    if x+1 < h_in_pixels:
        southN = curU[x+1, y]
        neighbours_sum += southN
    else:
        neighbours -= 1

    if y-1 >= 0:
        westN = curU[x, y-1]
        neighbours_sum += westN
    else:
        neighbours -= 1   
        
    return neighbours_sum, neighbours

In [67]:
def upd_grid(image_arr):
    global h_in_pixels, w_in_pixels
    global phi, epsilon, f, q, dt, du
    
    cur_u_grid = image_arr[:, :, 0]
    cur_v_grid = image_arr[:, :, 1]
    illum = image_arr[:, :, 2]
    new_u_grid = np.zeros((h_in_pixels, w_in_pixels))
    new_v_grid = np.zeros((h_in_pixels, w_in_pixels))
    sig_fig = 6

    for x in range(h_in_pixels):
        for y in range(w_in_pixels):
            u = cur_u_grid[x, y]
            v = cur_v_grid[x, y]
            if illum[x, y] != 0:
               new_u_grid[x, y] = 0
               new_v_grid[x, y] = 0
               continue

            neighbours_sum, neighbours = find_neighbours_sum(cur_u_grid, x, y)

            laplacian = du * ( (neighbours_sum - neighbours * cur_u_grid[x, y]) / 0.0625)
            
            # local concentrations of activator
            new_u_grid[x, y] = u + ( ((1.0/epsilon) * (u - np.square(u) - ((f*v + phi)*(u - q)/(u + q)))) + laplacian ) * dt
            
            # local concentrations of inhibitor
            new_v_grid[x, y] = v + ( u - v ) * dt

            # # clamp
            # new_u_grid[x, y] = clamp(0.0, 1.0, new_u_grid[x, y])
            # new_v_grid[x, y] = clamp(0.0, 1.0, new_v_grid[x, y])

    np.copyto(cur_u_grid, new_u_grid)
    np.copyto(cur_v_grid, new_v_grid)
    illum = image_arr[:, :, 2]
    new_image_arr = np.stack([cur_u_grid, cur_v_grid, illum], axis = -1)
    return new_image_arr

In [68]:
class VideoWriter:
  def __init__(self, filename, fps=30.0, **kw):
    self.writer = None
    self.params = dict(filename=filename, fps=fps, **kw)

  def add(self, img):
    img = np.asarray(img)
    if self.writer is None:
      h, w = img.shape[:2]
      self.writer = FFMPEG_VideoWriter(size=(w, h), **self.params)
    if img.dtype in [np.float32, np.float64]:
      img = np.uint8(img.clip(0, 1)*255)
    if len(img.shape) == 2:
      img = np.repeat(img[..., None], 3, -1)
    self.writer.write_frame(img)

  def close(self):
    if self.writer:
      self.writer.close()

  def __enter__(self):
    return self

  def __exit__(self, *kw):
    self.close()


In [69]:
def square_stimulus(cur_u_grid, y, x):
    global vesicle_size
    half_vesicle=vesicle_size//2 -1
    #  stim 4 at x, y locations!!
    x=(vesicle_size-1)*x + half_vesicle
    y=(vesicle_size-1)*y + half_vesicle
    cur_u_grid[x][y] = 1
    cur_u_grid[x+1][y+1] = 1
    cur_u_grid[x+1][y] = 1
    cur_u_grid[x][y+1] = 1
    return cur_u_grid

In [70]:
def makegrid(slit_len):
    global w_in_pixels, h_in_pixels, w_in_vesicles, h_in_vesicles
    half_vesicle=(vesicle_size-1)//2
    grid = np.zeros((h_in_pixels, w_in_pixels))
    for i in range(w_in_pixels):
        grid[0, i] = 1
        grid[h_in_pixels-1, i] = 1
    
    for i in range(h_in_pixels):
        grid[i, 0] = 1
        grid[i, w_in_pixels-1] = 1
    
    for h in range(1, h_in_vesicles+1):
        for w in range(1, w_in_vesicles+1):
            top = (h-1)*vesicle_size - (h-1-1) -1
            left = (w-1)*vesicle_size - (w-1-1) -1
            bottom = h*vesicle_size - (h-1) -1
            right = w*vesicle_size - (w-1) -1
            
            if right != w_in_pixels:
                for i in range(top+1, bottom):
                    middle = top+1 + half_vesicle
                    if i in range(middle - slit_len//2, middle + (slit_len - slit_len//2)):
                        pass
                    else:     
                        grid[i, right] = 1
            
            if bottom != h_in_pixels:
                for i in range(left+1, right):
                    middle = left+1 + half_vesicle
                    if i in range(middle - slit_len//2, middle + (slit_len - slit_len//2)):
                        pass
                    else:    
                        grid[bottom, i] = 1
            
            if right!=w_in_pixels and bottom!=h_in_pixels:
                grid[bottom, right] = 1
                if w==1:
                    grid[bottom, left] = 1
                if h==1:
                    grid[top, right] = 1
    return grid

In [71]:
def setgap(w, h, rightside: bool, slit_len):
    global illum
    w+=1
    h+=1
    top = (h-1)*vesicle_size - (h-1-1) -1
    left = (w-1)*vesicle_size - (w-1-1) -1
    bottom = h*vesicle_size - (h-1) -1
    right = w*vesicle_size - (w-1) -1
    
    half_vesicle=(vesicle_size-1)//2
    
    # on the right
    if rightside:
        for i in range(top+1, bottom):
            middle = top+1 + half_vesicle
            if i in range(middle - slit_len//2, middle + (slit_len - slit_len//2)):
                val = 0
            else:     
                val = 1
            illum[i, right] = val
                
    # on the bottom
    else:
        for i in range(left+1, right):
            middle = left+1 + half_vesicle
            if i in range(middle - slit_len//2, middle + (slit_len - slit_len//2)):
                val = 0
            else:    
                val = 1
            illum[bottom, i] = val

In [126]:
# [0.05, 0.08] is good (adamatzky2017fredkin, adamatzky2018street)
#  phi = 0.72
phi = 0.0745
# phi = 0.0745
epsilon = 0.0243; f = 1.4; q = 0.002; dt = 0.001; du = 0.45
h_in_vesicles = 2
w_in_vesicles = 3
vesicle_size = 16
slit_len = 0
selective_gap =8

iterations = 6000
record_every_n = 150
record_start_frame = 0

h_in_pixels = h_in_vesicles*vesicle_size - (h_in_vesicles-1) -1
w_in_pixels = w_in_vesicles*vesicle_size - (w_in_vesicles-1) -1

# ------------ main ------------ #
os.environ['FFMPEG_BINARY'] = 'ffmpeg'
# input_file=("./input_pics/vesicle_input_"+str(w_in_pixels)+"x"+str(h_in_pixels)+".png")
cur_u_grid = np.zeros((h_in_pixels, w_in_pixels)); cur_v_grid = np.zeros((h_in_pixels, w_in_pixels)); illum = makegrid(slit_len)

setgap(0, 0, True, selective_gap)
setgap(1, 0, True, selective_gap)
setgap(1, 0, False, selective_gap)

# stimulus 4 adjacent pixels
cur_u_grid = square_stimulus(cur_u_grid, 0, 0)
# cur_u_grid = square_stimulus(cur_u_grid, 2, 0)

image_arr = np.stack([cur_u_grid, cur_v_grid, illum], axis = -1)
image_arr = upd_grid(image_arr)
print("shape of image array:", image_arr.shape)

shape of image array: (58, 87, 3)


In [127]:
out_fn = "output.mp4"
Path(out_fn).touch()
with VideoWriter(out_fn) as vid:
    for i in tqdm.trange(iterations):
        if i > record_start_frame:
            if i%record_every_n == 0:
                vid.add(image_arr)
        image_arr = upd_grid(image_arr)

100%|██████████| 6000/6000 [02:14<00:00, 44.65it/s]


In [128]:
from IPython.display import display, Video, HTML
display_mult=5
display_width=w_in_pixels*display_mult
display_height=h_in_pixels*display_mult
video_html = f'<video src="{out_fn}" width="{display_width}" height="{display_height}" controls type="video/mp4" autoplay loop></video>'
display(HTML(video_html))